# CDR3 phenotype (Kawajiri et al)
- K. Ishigaki
- Follow-up scripts of 02_CDR3_phenotype-step1.txt output files

In [10]:
library(magrittr)
library(data.table)
library(ggsci)
library(ggplot2)

## Basic data

In [11]:
#all IMGT positions
all_imgt_pos <- c(
    "P104","P105","P106","P107","P108","P109","P110","P111",
    "P111.1","P112.2","P112.1",
    "P112","P113","P114","P115","P116","P117","P118")
length(all_imgt_pos)

[1] 18

In [12]:
all_aa <- c("A", "C", "D", "E", "F", "G", "H", "I", "K", "L", 
            "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y")
length(all_aa)

[1] 20

In [13]:
#Custom function to align CDR3 AA seq to IMGT
align_imgt <- function(CDR3){
    AA <- unlist(strsplit(CDR3, ""));
    N_AA_fow <- nchar(CDR3) %/% 2 + nchar(CDR3) %% 2;
    N_AA_na <- 18 - nchar(CDR3);
    AAmod <- c(
        AA[1:(N_AA_fow)],
        rep("NA",N_AA_na),
        AA[(N_AA_fow + 1):nchar(CDR3)]
    );
    names(AAmod) <- all_imgt_pos
    return(t(AAmod))
}

## CDR3 mid-position amino acid usage ratio

In [14]:
dd="/Users/kazu/Documents/Log/TCR_tohoku/analysis/2022-11-10/CDR3/tohoku_v2"
   #change the path as needed
   #the path to the files generated by 02_CDR3_phenotype-step1.txt
names=dir(dd)
names=grep("\\.productive.dat.gz",names,value=T)
names=gsub("\\.productive.dat.gz","",names)
names=setdiff(names,"Overview")
show(names)

[1] "AF1"  "AF2"  "AF3"  "SPF1" "SPF2" "SPF3"


In [15]:
res <- data.frame()
for(name in names){
    file=paste0(dd,"/",name,".productive.dat.gz")
    d1<-read.table(file,header=T,sep="\t")
    colnames(d1) <- c("Freq","CDR3","Vgene","Jgene")
    d1$Length <- nchar(as.character(d1$CDR3)) 
    d1 <- subset(d1,Length >=12 & Length <=18)
    
    #Add imgt position data to tdata_qc
    tdata_imgt_pos <- t(sapply(as.character(d1$CDR3),align_imgt))
    colnames(tdata_imgt_pos) <- all_imgt_pos
    
    d2 <- data.frame(d1,tdata_imgt_pos)
    d3 <- d2[,c("P108","P109","P110","P111","P111.1","P112.2","P112.1","P112"),]
    
    midAA <- t(apply(d3,1,function(x){
        x <- as.character(x);
        x <- x[x!="NA"]
        TB <- table(x)
        Ratio <- as.numeric(TB)/length(x)
        names(Ratio) <- names(TB)
        Ratio <- Ratio[all_aa]
        Ratio[is.na(Ratio)] <- 0
        return(Ratio)
    }))
    colnames(midAA) <- all_aa
    
    #calculate mean of each AA
    mean_ratio <- apply(midAA,2,function(x){mean(x)})
    out <- as.data.frame(t(mean_ratio))
    out <- data.frame(name,out)
    res <- rbind(res,out)
}
res

name,A,C,D,E,F,G,H,I,K,⋯,M,N,P,Q,R,S,T,V,W,Y
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AF1,0.06081684,0.0002709159,0.09431741,0.02841651,0.01312452,0.2651937,0.010278744,0.01739620,0.005391577,⋯,0.003676822,0.03407930,0.04441519,0.06244943,0.07209215,0.07170867,0.05975865,0.02273467,0.03673050,0.01373343
AF2,0.05842640,0.0003641829,0.09819331,0.03079792,0.01335642,0.2626202,0.010190082,0.01681155,0.005538777,⋯,0.003575934,0.03379601,0.04367154,0.06127700,0.07728352,0.07175692,0.05904220,0.02232097,0.03542367,0.01316233
AF3,0.06252887,0.0002693735,0.09647588,0.03073778,0.01255418,0.2618129,0.009753276,0.01712203,0.006551980,⋯,0.003639945,0.03416352,0.04304446,0.06394670,0.07584611,0.06897131,0.05854522,0.02073456,0.03532248,0.01454029
SPF1,0.06199297,0.0002478827,0.09487338,0.03112421,0.01371839,0.2573897,0.009720779,0.01880772,0.006110587,⋯,0.003290332,0.03536424,0.04470954,0.06482117,0.07616529,0.06994186,0.06171233,0.02128546,0.03223165,0.01292623
SPF2,0.06116829,0.0003145794,0.09452634,0.03108168,0.01372325,0.2634072,0.009664086,0.01935901,0.005814657,⋯,0.003358165,0.03461909,0.04465666,0.06575143,0.07501450,0.06917331,0.06156560,0.02120250,0.03136151,0.01214784
SPF3,0.06148463,0.0002993301,0.09355749,0.02991976,0.01330085,0.2645923,0.010099151,0.02022705,0.005646549,⋯,0.003405216,0.03474243,0.04273771,0.06499887,0.07626493,0.07172810,0.06049792,0.02098596,0.03040608,0.01163031


In [16]:
#OFILE="data/Tohoku_v2.rds"
#saveRDS(res, file =OFILE)